## Generate Object Information

### Setup

In [1]:
import json
from pathlib import Path
import typing
from pprint import pprint

In [2]:
rules = json.loads(Path("acquireRules.json").read_text())
resourceRules = rules["resources"]
liquidRules = rules["liquids"]
allObjectInfo = {}

### Helper Functions

In [3]:
def replacePlaceholders(text: str, replacements: dict[str,str]) -> str:
    """Replace placeholders in a string with the corresponding values."""
    
    for placeholder, value in replacements.items():
        text = text.replace(placeholder, value)
    
    return text

In [4]:
def recipeToText(recipe: dict) -> str:
    """Convert a recipe to an info string."""
    items = list(recipe.items())
    if len(items) == 1:
        return f"{items[0][1]} {items[0][0]}"
    return ", ".join([f"{v} {k}{'s' if v > 1 else ''}" for k, v in items[:-1]]) \
        + f" and {items[-1][1]} {items[-1][0]}{'s' if items[-1][1] > 1 else ''}"

### Generation Functions

In [5]:
def generateResourceInfo(data: dict[str, dict[str,typing.Any]], rules: dict[str: dict[str,str]]) -> dict[str,list[str]]:
    """Generate lists of strings forming the resource information.

    Args:
        data (dict[str, dict[str,typing.Any]]): resources data
        rules (dict[str, dict[str,str]]): acquisition rules

    Returns:
        dict[str,list[str]]: object information for each resource
    """
    
    unique: dict[str, str] = rules["unqiue"]
    categorical: dict[str, str] = rules["rules"]["category"]
    starts: dict[str, str] = rules["rules"]["startswith"]
    ends: dict[str, str] = rules["rules"]["endswith"]
    contains: dict[str, str] = rules["rules"]["contains"]
    
    objectInformation: dict[str, list[str]] = {}
    
    for objName,d in data.items():
        info: list[str] = []
        category = d["category"]
        
        info.append(f"Category: {category}")
        info.append(f"Value: {d['value']}")
        info.append(f"Weight: {d['weight']}")
        if (category == "FOOD"):
            info.append(f"Nutritional Value: {d['general1']}")
        
        # placeholder replacement values
        name = objName
        name1 = objName if objName.find("-") == -1 else objName.split("-")[0]
        name2 = objName if objName.find("-") == -1 else objName.split("-")[1]
        article = "an" if objName[0] in "AEIOU" else "a"
        article1 = "an" if name1[0] in "AEIOU" else "a"
        article2 = "an" if name2[0] in "AEIOU" else "a"
        replacements = {
            "@A": article,
            "@N": name
        }
        replacements1 = {
            "@A": article1,
            "@N": name1
        }
        replacements2 = {
            "@A": article2,
            "@N": name2
        }
    
        if objName in unique:
            info.append(f"Acquire: {unique[objName]}")
        elif objName in categorical:
            info.append(f"Acquire: {replacePlaceholders(categorical[objName], replacements2)}")
        else:
            found = False
            for key in starts.keys():
                if objName.startswith(key):
                    info.append(f"Acquire: {replacePlaceholders(starts[key], replacements2)}")
                    found = True
                    break
        
            if not found:
                for key in ends.keys():
                    if objName.endswith(key):
                        info.append(f"Acquire: {replacePlaceholders(ends[key], replacements1)}")
                        found = True
                        break
            
            if not found:
                for key in contains.keys():
                    if key in objName:
                        info.append(f"Acquire: {replacePlaceholders(contains[key], replacements)}")
                        break
                    
            if not found:
                raise ValueError(f"Could not find acquisition rule for {objName}")
        
        objectInformation[objName] = info
    
    return objectInformation

In [6]:
def generateCraftableResourceInfo(data: dict[str, dict[str,typing.Any]]) -> dict[str,list[str]]:
    """Generate lists of strings forming the craftable resource information.

    Args:
        data (dict[str, dict[str,typing.Any]]): crafable resources data

    Returns:
        dict[str,list[str]]: object information for each resource
    """
    
    objectInformation: dict[str, list[str]] = {}
    
    for objName,d in data.items():
        info: list[str] = []
        category = d["category"]
        
        info.append(f"Category: {category}")
        info.append(f"Value: {d['value']}")
        info.append(f"Weight: {d['weight']}")
        info.append(f"Acquire: Craft from {recipeToText(d['recipe'])}")
        
        objectInformation[objName] = info
    
    return objectInformation    

In [7]:
def generateContainerInfo(data: dict[str, dict[str,typing.Any]]) -> dict[str,list[str]]:
    """Generate lists of strings forming the container information.

    Args:
        data (dict[str, dict[str,typing.Any]]): container data

    Returns:
        dict[str,list[str]]: object information for each container
    """
    
    objectInformation: dict[str, list[str]] = {}
    
    for objName,d in data.items():
        info: list[str] = []
        category = "CONTAINER"
        
        info.append(f"Category: {category}")
        info.append(F"Capacity: {d['amount']}")
        info.append(f"Value: {d['value']}")
        info.append(f"Weight: {d['weight']}")
        info.append(f"Acquire: Craft from {recipeToText(d['recipe'])}")
        
        objectInformation[objName] = info
    
    return objectInformation

In [8]:
def generateMachineInfo(data: dict[str, dict[str,typing.Any]]) -> dict[str,list[str]]:
    """Generate lists of strings forming the machine information.

    Args:
        data (dict[str, dict[str,typing.Any]]): machine data

    Returns:
        dict[str,list[str]]: object information for each machine
    """
    
    objectInformation: dict[str, list[str]] = {}
    
    for objName,d in data.items():
        info: list[str] = []
        category = "MACHINE"
        
        info.append(f"Category: {category}")
        info.append(f"Value: {d['value']}")
        info.append(f"Weight: {d['weight']}")
        info.append(f"Acquire: Craft from {recipeToText(d['recipe'])}")
        if len(d['fuel']) == 0:
            info.append("Fuel: Not required")
        elif len(d['fuel']) == 1:
            info.append(f"Fuel: {d['fuel'][0]} required")
        else:
            info.append("Fuel: " + ", ".join(d['fuel'][:-1])+f" or {d['fuel'][-1]} required")
        
        objectInformation[objName] = info
    
    return objectInformation

In [9]:
def generateStoragesInfo(data: dict[str, dict[str,typing.Any]]) -> dict[str,list[str]]:
    """Generate lists of strings forming the storage information.

    Args:
        data (dict[str, dict[str,typing.Any]]): storage data

    Returns:
        dict[str,list[str]]: object information for each storage
    """
    
    objectInformation: dict[str, list[str]] = {}
    
    for objName,d in data.items():
        info: list[str] = []
        category = "STORAGE"
        
        info.append(f"Category: {category}")
        info.append(f"Value: {d['value']}")
        info.append(f"Weight: {d['weight']}")
        info.append(f"Acquire: Craft from {recipeToText(d['recipe'])}")
        info.append(f"Carryable: {'Yes' if d['carry'] else 'No'}")
        info.append(f"Capacity: {d['capacity']}")
        
        objectInformation[objName] = info
    
    return objectInformation

In [10]:
def generateToolsInfo(data: dict[str, dict[str,typing.Any]]) -> dict[str,list[str]]:
    """Generate lists of strings forming the tool information.

    Args:
        data (dict[str, dict[str,typing.Any]]): tool data

    Returns:
        dict[str,list[str]]: object information for each tool
    """
    
    objectInformation: dict[str, list[str]] = {}
    
    for objName,d in data.items():
        info: list[str] = []
        category = "TOOL"
        
        info.append(f"Category: {category}")
        info.append(f"Value: {d['value']}")
        info.append(f"Weight: {d['weight']}")
        info.append(f"Acquire: Craft from {recipeToText(d['recipe'])}")
        info.append(f"Durability: {'No limit' if d['durability'] < 0 else d['durability']}")
        
        objectInformation[objName] = info
    
    return objectInformation

In [11]:
def generateWeaponsInfo(data: dict[str, dict[str,typing.Any]]) -> dict[str,list[str]]:
    """Generate lists of strings forming the weapon information.

    Args:
        data (dict[str, dict[str,typing.Any]]): weapon data

    Returns:
        dict[str,list[str]]: object information for each weapon
    """
    
    objectInformation: dict[str, list[str]] = {}
    
    for objName,d in data.items():
        info: list[str] = []
        category = "WEAPON"
        
        info.append(f"Category: {category}")
        info.append(f"Value: {d['value']}")
        info.append(f"Weight: {d['weight']}")
        info.append(f"Acquire: Craft from {recipeToText(d['recipe'])}")
        info.append(f"Durability: {'No limit' if d['durability'] < 0 else d['durability']}")
        info.append(f"Melee Damage Ratio: {d['m-damage']}")
        info.append(f"Ranged Damage Ratio: {d['r-damage']}")
        info.append(f"Ammo Type: {d['ammo'].capitalize()}")
        
        objectInformation[objName] = info
    
    return objectInformation

In [12]:
def generateLiquidsInfo(data: dict[str, dict[str,typing.Any]], rules: dict[str: dict[str,str]]) -> dict[str,list[str]]:
    """Generate lists of strings forming the liquid information.

    Args:
        data (dict[str, dict[str,typing.Any]]): liquid data
        rules (dict[str, dict[str,str]]): acquisition rules

    Returns:
        dict[str,list[str]]: object information for each liquid
    """
    
    unique: dict[str, str] = rules["unqiue"]
    categorical: dict[str, str] = rules["rules"]["category"]
    starts: dict[str, str] = rules["rules"]["startswith"]
    ends: dict[str, str] = rules["rules"]["endswith"]
    contains: dict[str, str] = rules["rules"]["contains"]
    
    objectInformation: dict[str, list[str]] = {}
    
    for objName,d in data.items():
        if objName == "empty":
            continue
        
        info: list[str] = []
        category = "LIQUID"
        
        info.append(f"Category: {category}")
        info.append(f"Value: {d['value']}")
        info.append(f"Weight: {d['weight']}")
        
        # placeholder replacement values
        name = objName
        name1 = objName if objName.find("-") == -1 else objName.split("-")[0]
        name2 = objName if objName.find("-") == -1 else objName.split("-")[1]
        article = "an" if objName[0] in "AEIOU" else "a"
        article1 = "an" if name1[0] in "AEIOU" else "a"
        article2 = "an" if name2[0] in "AEIOU" else "a"
        replacements = {
            "@A": article,
            "@N": name
        }
        replacements1 = {
            "@A": article1,
            "@N": name1
        }
        replacements2 = {
            "@A": article2,
            "@N": name2
        }
    
        if objName in unique:
            info.append(f"Acquire: {unique[objName]}")
        elif objName in categorical:
            info.append(f"Acquire: {replacePlaceholders(categorical[objName], replacements2)}")
        else:
            found = False
            for key in starts.keys():
                if objName.startswith(key):
                    info.append(f"Acquire: {replacePlaceholders(starts[key], replacements2)}")
                    found = True
                    break
        
            if not found:
                for key in ends.keys():
                    if objName.endswith(key):
                        info.append(f"Acquire: {replacePlaceholders(ends[key], replacements1)}")
                        found = True
                        break
            
            if not found:
                for key in contains.keys():
                    if key in objName:
                        info.append(f"Acquire: {replacePlaceholders(contains[key], replacements)}")
                        break
                    
            if not found:
                raise ValueError(f"Could not find acquisition rule for {objName}")
        
        info.append(f"Hunger: {d['hunger']}")
        info.append(f"Thirst: {d['thirst']}")
        info.append(f"Health: {d['hp']}")
        
        objectInformation[objName] = info
    
    return objectInformation

### Load Data

In [13]:
resourceData: dict = json.loads(Path("../data/resources.json").read_text())
craftableResourceData: dict = json.loads(Path("../data/craftableResources.json").read_text())
containerData: dict = json.loads(Path("../data/containers.json").read_text())
toolsData: dict = json.loads(Path("../data/tools.json").read_text())
weaponsData: dict = json.loads(Path("../data/weapons.json").read_text())
storagesData: dict = json.loads(Path("../data/storages.json").read_text())
machinesData: dict = json.loads(Path("../data/machines.json").read_text())
liquidsData: dict = json.loads(Path("../data/liquids.json").read_text())

### Generate

In [14]:
PRINTING = False

In [15]:
resourceInfo = generateResourceInfo(resourceData, resourceRules)
assert len(resourceData) == len(resourceInfo), "Missing resource info"
if PRINTING:
    pprint(resourceInfo)
allObjectInfo.update(resourceInfo.items())

In [16]:
craftableResourceInfo = generateCraftableResourceInfo(craftableResourceData)
assert len(craftableResourceData) == len(craftableResourceInfo), "Missing craftable resource info"
if PRINTING:
    pprint(craftableResourceInfo)
allObjectInfo.update(craftableResourceInfo.items())

In [17]:
containerInfo = generateContainerInfo(containerData)
assert len(containerData) == len(containerInfo), "Missing container info"
if PRINTING:
    pprint(containerInfo)
allObjectInfo.update(containerInfo.items())

In [18]:
machineInfo = generateMachineInfo(machinesData)
assert len(machinesData) == len(machineInfo), "Missing machine info"
if PRINTING:
    pprint(machineInfo)
allObjectInfo.update(machineInfo.items())

In [19]:
storagesInfo = generateStoragesInfo(storagesData)
assert len(storagesData) == len(storagesInfo), "Missing storage info"
if PRINTING:
    pprint(storagesInfo)
allObjectInfo.update(storagesInfo.items())

In [20]:
toolsInfo = generateToolsInfo(toolsData)
assert len(toolsData) == len(toolsInfo), "Missing tool info"
if PRINTING:
    pprint(toolsInfo)
allObjectInfo.update(toolsInfo.items())

In [21]:
weaponsInfo = generateWeaponsInfo(weaponsData)
assert len(weaponsData) == len(weaponsInfo), "Missing weapon info"
if PRINTING:
    pprint(weaponsInfo)
allObjectInfo.update(weaponsInfo.items())

In [22]:
liquidsInfo = generateLiquidsInfo(liquidsData, liquidRules)
assert len(liquidsData) == len(liquidsInfo) + 1, "Missing liquid info"
if PRINTING:
    pprint(liquidsInfo)
allObjectInfo.update(liquidsInfo.items())

### Save

In [23]:
print(len(allObjectInfo), "info items generated")
_ = Path("../infodata/object_info.json").write_text(json.dumps(dict(sorted(allObjectInfo.items(), key = lambda x: x[0])), indent=4))

88 info items generated
